![Egeria Logo](https://raw.githubusercontent.com/odpi/egeria/master/assets/img/ODPi_Egeria_Logo_color.png)

### Performance Suite Results
# Crux Plugin Repository Connector

## Calculating medians

This notebook simply calculates the medians for every method executed by the PTS.

## Setup

### Results locations

Locations for the results (see subdirectories in the same location where this notebook resides to review the raw results themselves):

In [1]:
results = [
    "pts-05-02",
    "pts-10-05",
    "pts-20-10",
    "pts-40-10"
]

### Parsing and analysis methods

The following defines methods necessary to parse and process the results, and must be run prior to the subsequent cells.

In [2]:
import os
import json
import pandas as pd
from IPython.display import display

def validateProfileResultsLocation(location):
    profile_details_location = location + os.path.sep + "profile-details"
    print("Validating profile-details location:", profile_details_location)
    if os.path.isdir(profile_details_location):
        print(" ... directory exists.")
    else:
        print(" ... ERROR: could not find this directory. Is the location specified correct?")

# Define the profile ordering
profile_order=[
    'Entity creation', 'Entity search', 'Relationship creation', 'Relationship search',
    'Entity classification', 'Classification search', 'Entity update', 'Relationship update',
    'Classification update', 'Entity undo', 'Relationship undo', 'Entity retrieval', 'Entity history retrieval',
    'Relationship retrieval', 'Relationship history retrieval', 'Entity history search', 'Relationship history search',
    'Graph queries', 'Graph history queries', 'Entity re-home', 'Relationship re-home', 'Entity declassify',
    'Entity re-type', 'Relationship re-type', 'Entity re-identify', 'Relationship re-identify',
    'Relationship delete', 'Entity delete', 'Entity restore', 'Relationship restore', 'Relationship purge',
    'Entity purge'
]

# Given a profileResult.requirementResults object, parse all of its positiveTestEvidence
# and group the results by methodName
def parseEvidence(df, repositoryName, requirementResults):
    if (requirementResults is not None and 'positiveTestEvidence' in requirementResults):
        print("Parsing evidence for:", requirementResults['name'], "(" + repositoryName + ")")
        data_array = []
        for evidence in requirementResults['positiveTestEvidence']:
            if ('methodName' in evidence and 'elapsedTime' in evidence):
                data = {
                    'repo': repositoryName,
                    'method_name': evidence['methodName'],
                    'elapsed_time': evidence['elapsedTime'],
                    'profile_name': requirementResults['name'],
                    'test_case_id': evidence['testCaseId'],
                    'assertion_id': evidence['assertionId']
                }
                data_array.append(data)
        df = df.append(pd.read_json(json.dumps(data_array), orient='records'), ignore_index=True)
    return df

# Given a profile detail JSON file, retrieve all of its profileResult.requirementResults[] objects
def parseRequirementResults(profileFile):
    with open(profileFile) as f:
        profile = json.load(f)
    # This first case covers files retrieved via API
    if ('profileResult' in profile and 'requirementResults' in profile['profileResult']):
        return profile['profileResult']['requirementResults']
    # This second case covers files created by the CLI client
    elif ('requirementResults' in profile):
        return profile['requirementResults']
    else:
        return None

# Retrieve a listing of all of the profile detail JSON files
def getAllProfiles(profileLocation):
    detailsLocation = profileLocation + os.path.sep + "profile-details"
    _, _, filenames = next(os.walk(detailsLocation))
    full_filenames = []
    for filename in filenames:
        full_filenames.append(detailsLocation + os.path.sep + filename)
    return full_filenames

# Parse all of the provided profile file's details into the provided dataframe
def parseProfileDetailsIntoDF(df, profileFile, qualifier):
    profileResults = parseRequirementResults(profileFile)
    if profileResults is not None:
        for result in profileResults:
            df = parseEvidence(df, qualifier, result)
    return df

# The results

## Parse all results

In [3]:
df1 = pd.DataFrame({'repo': [], 'method_name': [], 'elapsed_time': [], 'profile_name': [], 'test_case_id': [], 'assertion_id': []})

for result_name in results:
    validateProfileResultsLocation(result_name)
    files = getAllProfiles(result_name)
    dfEnv = None
    for profile_file in files:
        df1 = parseProfileDetailsIntoDF(df1, profile_file, result_name)

Validating profile-details location: pts-05-02/profile-details
 ... directory exists.
Parsing evidence for: Relationship undo (pts-05-02)
Parsing evidence for: Relationship history search (pts-05-02)
Parsing evidence for: Classification update (pts-05-02)
Parsing evidence for: Entity update (pts-05-02)
Parsing evidence for: Graph queries (pts-05-02)
Parsing evidence for: Relationship purge (pts-05-02)
Parsing evidence for: Entity history retrieval (pts-05-02)
Parsing evidence for: Entity re-identify (pts-05-02)
Parsing evidence for: Relationship update (pts-05-02)
Parsing evidence for: Entity creation (pts-05-02)
Parsing evidence for: Relationship creation (pts-05-02)
Parsing evidence for: Entity history search (pts-05-02)
Parsing evidence for: Entity classification (pts-05-02)
Parsing evidence for: Entity retrieval (pts-05-02)
Parsing evidence for: Relationship delete (pts-05-02)
Parsing evidence for: Relationship history retrieval (pts-05-02)
Parsing evidence for: Relationship search

## Print all results

In [4]:
header  = "Profile | Method"
breaker = "---|---"
for result_name in results:
    header  += " | " + result_name
    breaker += "|---"
print(header)
print(breaker)
for profile_name in profile_order:
    groups = df1[df1['profile_name'] == profile_name].groupby(['repo', 'method_name'], as_index=False).median()
    methods = sorted(groups['method_name'].unique())
    grouped_results = []
    for result in results:
        grouped_results.append(groups[groups['repo'] == result])
    index = 0
    for method in methods:
        row = " ... | "
        if index == 0:
            row = profile_name + " | "
        row += method
        for result in grouped_results:
            value = result.loc[result['method_name'] == method, 'elapsed_time']
            if value.empty:
                row += " | --"
            else:
                row += " | " + str(value.values[0])
        print(row)
        index += 1

Profile | Method | pts-05-02 | pts-10-05 | pts-20-10 | pts-40-10
---|---|---|---|---|---
Entity creation | addEntity | 95.0 | 93.0 | 94.0 | 93.0
 ... | saveEntityReferenceCopy | 88.0 | 89.0 | 91.0 | 90.0
Entity search | findEntities | 242.0 | 677.0 | 1167.0 | 2140.0
 ... | findEntitiesByProperty | 208.0 | 424.0 | 823.0 | 2018.0
 ... | findEntitiesByPropertyValue | 79.0 | 176.0 | 273.5 | 453.5
Relationship creation | addRelationship | 96.0 | 97.0 | 97.0 | 95.0
 ... | saveRelationshipReferenceCopy | 98.0 | 98.0 | 98.0 | 96.0
Relationship search | findRelationships | 462.0 | 1186.0 | 2203.5 | 5195.0
 ... | findRelationshipsByProperty | 354.0 | 760.0 | 1458.0 | 3554.0
 ... | findRelationshipsByPropertyValue | 58.5 | 99.0 | 206.5 | 363.5
Entity classification | classifyEntity | 126.0 | 139.5 | 132.0 | 129.0
 ... | saveClassificationReferenceCopy | 116.0 | 129.0 | 125.0 | 120.0
Classification search | findEntitiesByClassification | 39.0 | 55.0 | 73.0 | 97.0
Entity update | reTypeEntity | 97.